In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import pandas as pd
import numpy as np
import os, string, re, csv


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 


# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

#for grammar
import spacy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


In [2]:
path = "./all-the-news"
df = pd.DataFrame()
for file in os.listdir(path):
    if file != "formatted.csv":
        print(path+"/"+file)
        dfnew = pd.read_csv(path+"/"+file)
        df = pd.concat([df, dfnew])
articles = df["content"].to_numpy()

def clean_text(txt):
    txt = txt.encode("utf8").decode("ascii",'ignore')
    txt = re.sub(' +',' ', txt)
    txt = txt.replace("\t", "").replace("\r", "")
    return txt 

def write_formatted_csv(article_list):
    with open(path+"/formatted.csv", mode="w", newline=None) as outfile:
        writer = csv.writer(outfile)
        for article in article_list:
            writer.writerow(article)



./all-the-news/articles1.csv
./all-the-news/articles2.csv
./all-the-news/articles3.csv


In [3]:
# write_formatted_csv(corpus)

In [4]:
# word_index = {}
# word_frequency_index = {}
# counter = 0
# for article in corpus:
#     try:
#         article = article.split(" ")
#         if len(article) > 1:
#             for word in article:
#                 word = word.lower()
#                 word = "".join([w for w in word if w not in string.punctuation])
#                 if word not in word_index:
#                     word_index[word] = counter
#                     counter += 1
#                 if word not in word_frequency_index:
#                     word_frequency_index[word] = 1
#                 else:
#                     word_frequency_index[word] += 1
#     except:
#         pass

In [5]:
#convert corpus into one string
articles = [clean_text(x) for x in articles if type(x) is str]

In [6]:
corpus = " ".join(articles)

In [7]:
vocab = sorted(set(corpus))

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus], dtype=np.uint8)

In [9]:
print(f"{corpus[:10]} - {text_as_int[:10]}")

WASHINGTON - [51 29 47 36 37 42 35 48 43 42]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(corpus)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [13]:
dataset = sequences.map(split_input_target)

In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.uint8, tf.uint8)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23296     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 91)            93275     
Total params: 5,363,547
Trainable params: 5,363,547
Non-trainable params: 0
_________________________________________________________________


In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 91) # (batch_size, sequence_length, vocab_size)


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 91)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.510748


In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
history = model.fit(dataset, epochs=100, steps_per_epoch=100, callbacks=[checkpoint_callback])

Epoch 1/100
 99/100 [============================>.] - ETA: 15:38 - loss: 4.51 - ETA: 11:50 - loss: 4.48 - ETA: 10:23 - loss: 4.33 - ETA: 9:29 - loss: 4.4981 - ETA: 8:52 - loss: 4.271 - ETA: 8:25 - loss: 4.154 - ETA: 8:05 - loss: 4.075 - ETA: 7:51 - loss: 4.009 - ETA: 7:38 - loss: 3.949 - ETA: 7:27 - loss: 3.885 - ETA: 7:16 - loss: 3.823 - ETA: 7:09 - loss: 3.763 - ETA: 7:01 - loss: 3.711 - ETA: 6:53 - loss: 3.666 - ETA: 6:46 - loss: 3.632 - ETA: 6:38 - loss: 3.602 - ETA: 6:31 - loss: 3.573 - ETA: 6:25 - loss: 3.546 - ETA: 6:18 - loss: 3.519 - ETA: 6:12 - loss: 3.497 - ETA: 6:06 - loss: 3.476 - ETA: 6:00 - loss: 3.458 - ETA: 5:54 - loss: 3.440 - ETA: 5:48 - loss: 3.425 - ETA: 5:43 - loss: 3.410 - ETA: 5:37 - loss: 3.396 - ETA: 5:32 - loss: 3.384 - ETA: 5:26 - loss: 3.371 - ETA: 5:20 - loss: 3.359 - ETA: 5:15 - loss: 3.348 - ETA: 5:10 - loss: 3.339 - ETA: 5:05 - loss: 3.329 - ETA: 4:59 - loss: 3.319 - ETA: 4:54 - loss: 3.309 - ETA: 4:49 - loss: 3.300 - ETA: 4:44 - loss: 3.291 - ETA: 4:3

 99/100 [============================>.] - ETA: 6:51 - loss: 2.276 - ETA: 6:47 - loss: 2.261 - ETA: 6:42 - loss: 2.253 - ETA: 6:38 - loss: 2.259 - ETA: 6:34 - loss: 2.258 - ETA: 6:30 - loss: 2.268 - ETA: 6:26 - loss: 2.266 - ETA: 6:22 - loss: 2.267 - ETA: 6:18 - loss: 2.268 - ETA: 6:14 - loss: 2.265 - ETA: 6:09 - loss: 2.265 - ETA: 6:05 - loss: 2.266 - ETA: 6:01 - loss: 2.263 - ETA: 5:57 - loss: 2.259 - ETA: 5:53 - loss: 2.260 - ETA: 5:49 - loss: 2.258 - ETA: 5:45 - loss: 2.258 - ETA: 5:40 - loss: 2.257 - ETA: 5:36 - loss: 2.255 - ETA: 5:32 - loss: 2.253 - ETA: 5:28 - loss: 2.252 - ETA: 5:24 - loss: 2.251 - ETA: 5:19 - loss: 2.249 - ETA: 5:15 - loss: 2.248 - ETA: 5:11 - loss: 2.246 - ETA: 5:07 - loss: 2.246 - ETA: 5:03 - loss: 2.243 - ETA: 4:59 - loss: 2.242 - ETA: 4:54 - loss: 2.241 - ETA: 4:50 - loss: 2.242 - ETA: 4:46 - loss: 2.240 - ETA: 4:42 - loss: 2.239 - ETA: 4:38 - loss: 2.238 - ETA: 4:33 - loss: 2.237 - ETA: 4:29 - loss: 2.235 - ETA: 4:25 - loss: 2.234 - ETA: 4:21 - loss: 2.2

 99/100 [============================>.] - ETA: 6:48 - loss: 1.947 - ETA: 6:45 - loss: 1.967 - ETA: 6:41 - loss: 1.952 - ETA: 6:37 - loss: 1.942 - ETA: 6:33 - loss: 1.935 - ETA: 6:28 - loss: 1.938 - ETA: 6:24 - loss: 1.936 - ETA: 6:20 - loss: 1.935 - ETA: 6:17 - loss: 1.935 - ETA: 6:13 - loss: 1.931 - ETA: 6:09 - loss: 1.930 - ETA: 6:04 - loss: 1.932 - ETA: 6:00 - loss: 1.932 - ETA: 5:56 - loss: 1.932 - ETA: 5:52 - loss: 1.930 - ETA: 5:48 - loss: 1.932 - ETA: 5:44 - loss: 1.931 - ETA: 5:40 - loss: 1.929 - ETA: 5:35 - loss: 1.928 - ETA: 5:31 - loss: 1.927 - ETA: 5:27 - loss: 1.927 - ETA: 5:23 - loss: 1.928 - ETA: 5:19 - loss: 1.927 - ETA: 5:14 - loss: 1.926 - ETA: 5:10 - loss: 1.926 - ETA: 5:06 - loss: 1.926 - ETA: 5:02 - loss: 1.925 - ETA: 4:58 - loss: 1.924 - ETA: 4:54 - loss: 1.922 - ETA: 4:50 - loss: 1.921 - ETA: 4:45 - loss: 1.921 - ETA: 4:41 - loss: 1.920 - ETA: 4:37 - loss: 1.919 - ETA: 4:33 - loss: 1.919 - ETA: 4:29 - loss: 1.918 - ETA: 4:25 - loss: 1.917 - ETA: 4:20 - loss: 1.9

 99/100 [============================>.] - ETA: 6:49 - loss: 1.704 - ETA: 6:47 - loss: 1.728 - ETA: 6:42 - loss: 1.723 - ETA: 6:38 - loss: 1.713 - ETA: 6:34 - loss: 1.722 - ETA: 6:29 - loss: 1.726 - ETA: 6:25 - loss: 1.723 - ETA: 6:21 - loss: 1.723 - ETA: 6:17 - loss: 1.721 - ETA: 6:13 - loss: 1.721 - ETA: 6:09 - loss: 1.721 - ETA: 6:05 - loss: 1.721 - ETA: 6:00 - loss: 1.720 - ETA: 5:58 - loss: 1.721 - ETA: 5:55 - loss: 1.721 - ETA: 5:50 - loss: 1.720 - ETA: 5:46 - loss: 1.720 - ETA: 5:42 - loss: 1.720 - ETA: 5:38 - loss: 1.719 - ETA: 5:33 - loss: 1.717 - ETA: 5:29 - loss: 1.717 - ETA: 5:25 - loss: 1.717 - ETA: 5:21 - loss: 1.718 - ETA: 5:16 - loss: 1.719 - ETA: 5:12 - loss: 1.718 - ETA: 5:08 - loss: 1.718 - ETA: 5:04 - loss: 1.715 - ETA: 5:00 - loss: 1.716 - ETA: 4:55 - loss: 1.716 - ETA: 4:51 - loss: 1.717 - ETA: 4:47 - loss: 1.717 - ETA: 4:43 - loss: 1.717 - ETA: 4:39 - loss: 1.714 - ETA: 4:34 - loss: 1.712 - ETA: 4:30 - loss: 1.712 - ETA: 4:26 - loss: 1.710 - ETA: 4:22 - loss: 1.7

 99/100 [============================>.] - ETA: 6:51 - loss: 1.659 - ETA: 6:46 - loss: 1.646 - ETA: 6:41 - loss: 1.642 - ETA: 6:37 - loss: 1.645 - ETA: 6:33 - loss: 1.631 - ETA: 6:29 - loss: 1.622 - ETA: 6:25 - loss: 1.615 - ETA: 6:21 - loss: 1.608 - ETA: 6:17 - loss: 1.612 - ETA: 6:13 - loss: 1.613 - ETA: 6:09 - loss: 1.608 - ETA: 6:05 - loss: 1.611 - ETA: 6:00 - loss: 1.615 - ETA: 5:56 - loss: 1.610 - ETA: 5:52 - loss: 1.608 - ETA: 5:48 - loss: 1.609 - ETA: 5:44 - loss: 1.606 - ETA: 5:40 - loss: 1.607 - ETA: 5:35 - loss: 1.609 - ETA: 5:31 - loss: 1.609 - ETA: 5:27 - loss: 1.608 - ETA: 5:23 - loss: 1.608 - ETA: 5:19 - loss: 1.608 - ETA: 5:15 - loss: 1.608 - ETA: 5:11 - loss: 1.606 - ETA: 5:06 - loss: 1.605 - ETA: 5:02 - loss: 1.606 - ETA: 4:58 - loss: 1.603 - ETA: 4:54 - loss: 1.602 - ETA: 4:50 - loss: 1.602 - ETA: 4:46 - loss: 1.601 - ETA: 4:42 - loss: 1.601 - ETA: 4:38 - loss: 1.601 - ETA: 4:34 - loss: 1.602 - ETA: 4:29 - loss: 1.601 - ETA: 4:25 - loss: 1.601 - ETA: 4:21 - loss: 1.6

...

 99/100 [============================>.] - ETA: 6:47 - loss: 1.238 - ETA: 6:43 - loss: 1.252 - ETA: 6:39 - loss: 1.236 - ETA: 6:35 - loss: 1.242 - ETA: 6:31 - loss: 1.233 - ETA: 6:27 - loss: 1.240 - ETA: 6:23 - loss: 1.239 - ETA: 6:19 - loss: 1.241 - ETA: 6:15 - loss: 1.240 - ETA: 6:11 - loss: 1.236 - ETA: 6:07 - loss: 1.233 - ETA: 6:03 - loss: 1.237 - ETA: 5:59 - loss: 1.240 - ETA: 5:55 - loss: 1.239 - ETA: 5:51 - loss: 1.237 - ETA: 5:46 - loss: 1.234 - ETA: 5:42 - loss: 1.236 - ETA: 5:38 - loss: 1.235 - ETA: 5:34 - loss: 1.235 - ETA: 5:30 - loss: 1.235 - ETA: 5:26 - loss: 1.234 - ETA: 5:22 - loss: 1.233 - ETA: 5:17 - loss: 1.234 - ETA: 5:13 - loss: 1.232 - ETA: 5:09 - loss: 1.231 - ETA: 5:05 - loss: 1.230 - ETA: 5:01 - loss: 1.229 - ETA: 4:57 - loss: 1.228 - ETA: 4:53 - loss: 1.227 - ETA: 4:49 - loss: 1.226 - ETA: 4:45 - loss: 1.227 - ETA: 4:40 - loss: 1.227 - ETA: 4:36 - loss: 1.227 - ETA: 4:32 - loss: 1.227 - ETA: 4:28 - loss: 1.227 - ETA: 4:24 - loss: 1.227 - ETA: 4:20 - loss: 1.2

 99/100 [============================>.] - ETA: 6:48 - loss: 1.219 - ETA: 6:43 - loss: 1.211 - ETA: 6:39 - loss: 1.209 - ETA: 6:34 - loss: 1.215 - ETA: 6:30 - loss: 1.223 - ETA: 6:26 - loss: 1.218 - ETA: 6:22 - loss: 1.215 - ETA: 6:18 - loss: 1.216 - ETA: 6:14 - loss: 1.216 - ETA: 6:10 - loss: 1.215 - ETA: 6:06 - loss: 1.215 - ETA: 6:02 - loss: 1.212 - ETA: 5:58 - loss: 1.210 - ETA: 5:54 - loss: 1.209 - ETA: 5:50 - loss: 1.211 - ETA: 5:46 - loss: 1.214 - ETA: 5:42 - loss: 1.216 - ETA: 5:38 - loss: 1.216 - ETA: 5:34 - loss: 1.217 - ETA: 5:29 - loss: 1.219 - ETA: 5:25 - loss: 1.220 - ETA: 5:21 - loss: 1.221 - ETA: 5:17 - loss: 1.221 - ETA: 5:13 - loss: 1.222 - ETA: 5:09 - loss: 1.222 - ETA: 5:06 - loss: 1.222 - ETA: 5:03 - loss: 1.223 - ETA: 4:59 - loss: 1.222 - ETA: 4:56 - loss: 1.223 - ETA: 4:52 - loss: 1.223 - ETA: 4:48 - loss: 1.222 - ETA: 4:44 - loss: 1.221 - ETA: 4:39 - loss: 1.222 - ETA: 4:35 - loss: 1.222 - ETA: 4:31 - loss: 1.222 - ETA: 4:27 - loss: 1.221 - ETA: 4:22 - loss: 1.2

 99/100 [============================>.] - ETA: 6:48 - loss: 1.208 - ETA: 6:44 - loss: 1.209 - ETA: 6:42 - loss: 1.199 - ETA: 6:37 - loss: 1.199 - ETA: 6:33 - loss: 1.201 - ETA: 6:28 - loss: 1.204 - ETA: 6:24 - loss: 1.206 - ETA: 6:20 - loss: 1.209 - ETA: 6:16 - loss: 1.210 - ETA: 6:11 - loss: 1.212 - ETA: 6:07 - loss: 1.215 - ETA: 6:03 - loss: 1.214 - ETA: 5:59 - loss: 1.214 - ETA: 5:55 - loss: 1.212 - ETA: 5:51 - loss: 1.211 - ETA: 5:47 - loss: 1.213 - ETA: 5:43 - loss: 1.213 - ETA: 5:38 - loss: 1.214 - ETA: 5:34 - loss: 1.214 - ETA: 5:30 - loss: 1.213 - ETA: 5:26 - loss: 1.214 - ETA: 5:22 - loss: 1.212 - ETA: 5:18 - loss: 1.210 - ETA: 5:14 - loss: 1.212 - ETA: 5:09 - loss: 1.213 - ETA: 5:05 - loss: 1.212 - ETA: 5:01 - loss: 1.212 - ETA: 4:57 - loss: 1.211 - ETA: 4:53 - loss: 1.210 - ETA: 4:49 - loss: 1.211 - ETA: 4:45 - loss: 1.210 - ETA: 4:40 - loss: 1.210 - ETA: 4:36 - loss: 1.209 - ETA: 4:32 - loss: 1.208 - ETA: 4:28 - loss: 1.209 - ETA: 4:24 - loss: 1.209 - ETA: 4:20 - loss: 1.2

 99/100 [============================>.] - ETA: 6:53 - loss: 1.218 - ETA: 6:47 - loss: 1.207 - ETA: 6:42 - loss: 1.215 - ETA: 6:37 - loss: 1.212 - ETA: 6:33 - loss: 1.211 - ETA: 6:28 - loss: 1.215 - ETA: 6:24 - loss: 1.213 - ETA: 6:19 - loss: 1.220 - ETA: 6:15 - loss: 1.218 - ETA: 6:11 - loss: 1.218 - ETA: 6:07 - loss: 1.219 - ETA: 6:03 - loss: 1.221 - ETA: 5:59 - loss: 1.220 - ETA: 5:55 - loss: 1.220 - ETA: 5:50 - loss: 1.219 - ETA: 5:46 - loss: 1.220 - ETA: 5:42 - loss: 1.222 - ETA: 5:38 - loss: 1.222 - ETA: 5:34 - loss: 1.221 - ETA: 5:30 - loss: 1.223 - ETA: 5:26 - loss: 1.223 - ETA: 5:22 - loss: 1.223 - ETA: 5:18 - loss: 1.223 - ETA: 5:14 - loss: 1.224 - ETA: 5:10 - loss: 1.224 - ETA: 5:06 - loss: 1.224 - ETA: 5:01 - loss: 1.224 - ETA: 4:57 - loss: 1.226 - ETA: 4:53 - loss: 1.224 - ETA: 4:49 - loss: 1.224 - ETA: 4:45 - loss: 1.224 - ETA: 4:41 - loss: 1.223 - ETA: 4:36 - loss: 1.225 - ETA: 4:32 - loss: 1.224 - ETA: 4:28 - loss: 1.225 - ETA: 4:24 - loss: 1.224 - ETA: 4:20 - loss: 1.2

KeyboardInterrupt: 

In [47]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23296     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 91)             93275     
Total params: 5,363,547
Trainable params: 5,363,547
Non-trainable params: 0
_________________________________________________________________


In [48]:
def generate_text(model, start_string):
   # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [49]:
print(generate_text(model, start_string=u"hello"))

hellon haunts several polls with their crime, she ability to be about: the current choices had tried to evaluate freedom, but also because the time I think it seems like it, You get themselves in this approva. A New York city filed by Pene Pand, said with O. C. The two good walls in secondbody, at a mission to prison for canapartic bis and promotes a wall for a moment campaign, has never been frustrations, ard others are talking for in order to support him expected to speak together.) As together, We were wedding on beyond North Carolina. Ordered the truth your it as stumbling almost one on that corone on top loans. Tesls before you arrested. Its 90 mune dashed to have offered possible rights and waits so offendes may never have everybody next morning was the only in others it was as if smart, Find have just dont work. Cerform the caused fah of clerians to be a resilient, the scientist s have struggled with ve their overtaskpect. ____ 9. Mr. Band. Mr. Kinbert, a family person, more tha